In [1]:
import numpy as np
import pandas as pd

import requests

from bs4 import BeautifulSoup

import json
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

Retrieve the html of the Wikipedia page

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Load the html into the soup object

In [3]:
soup = BeautifulSoup(source, 'html.parser')

We only want to process the table tag

In [4]:
table = soup.find('table', class_='wikitable sortable')

load the data of the table into a python list

In [5]:
data = []
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

delete the first element in the list as the list is empty

In [6]:
del data[0]

replace the 'Not assigned' Neighborhood with its Borough

In [7]:
for i in range(len(data)):
    if 'Not assigned' in data[i-1][2]:
        data[i-1][2] = data[i-1][1]

Load the list into pandas dataframe

In [8]:
df = pd.DataFrame(data)

Give the columns names.

In [9]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

Discard any Borough with 'Not assigned' values

In [10]:
df = df[df['Borough'] != 'Not assigned']

Combine Neighborhood that have same PostalCode into a row.

In [11]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))

In [12]:
df = df.reset_index()

Load the Geospatial Coordinate csv file into a dataframe.

In [13]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename the column name te ease the merging of the two dataframe.

In [14]:
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

Merge the two dataframe according its Postal Code. 

In [15]:
df_toronto = pd.merge(df, geo_df, on='PostalCode', how='inner')

In [16]:
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [17]:
import folium

from geopy.geocoders import Nominatim

Get the latitude and longitude values of Toronto using Geopy library

In [18]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograhical coordinate of Toronto City are {}, {}'.format(latitude, longitude))

C:\Users\Luqman Hakim\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograhical coordinate of Toronto City are 43.653963, -79.387207


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, postal_code, borough, neighborhoods in zip(df_toronto['Latitude'], 
                                                         df_toronto['Longitude'], 
                                                         df_toronto['PostalCode'], 
                                                         df_toronto['Borough'], 
                                                         df_toronto['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhoods, postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Explore the Neighborhoods in Toronto

In [20]:
CLIENT_ID = 'UUNFJ1AY3XVOH5CNC1HGZSMU3LGUVWIVAYQ4DDKJZAUESURA'
CLIENT_SECRET = 'GUM2NDBB2MEJOWJIQYCCYDYZGEPOJGSMY1GLU2CLPKDTYI1T'
VERSION = '20181110'

Create a function to get the top venues that are in each postal code in Toronto

In [21]:
def getNearbyVenues(postal_codes, latitudes, longitudes, radius=500, limit=10):
    venues_list=[]
    for postal_code, lat, lng in zip(postal_codes, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# Default values for radius (500 meters) and limit (10 venues maximum)
toronto_venues = getNearbyVenues(postal_codes = df_toronto['PostalCode'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude'])

This gives us a new dataframe called toronto_venues

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(700, 7)


,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1C,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,M1C,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [24]:
toronto_venues.groupby('PostalCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,2,2,2,2,2,2
M1C,3,3,3,3,3,3
M1E,6,6,6,6,6,6
M1G,4,4,4,4,4,4
M1H,8,8,8,8,8,8
M1J,2,2,2,2,2,2
M1K,7,7,7,7,7,7
M1L,9,9,9,9,9,9
M1M,2,2,2,2,2,2


In [25]:
print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 180 uniques categories


### Analyze Each Postal Code

In [26]:
# one hot encoding for venues
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal codes column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(700, 181)

In [28]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
toronto_grouped.shape

(101, 181)

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Yoga Studio,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,M1C,Bar,Moving Target,History Museum,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
2,M1E,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant
3,M1G,Coffee Shop,Mexican Restaurant,Korean Restaurant,Garden Center,Deli / Bodega,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
4,M1H,Lounge,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Yoga Studio,Department Store


In [32]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 2, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1])

In [33]:
toronto_merged = pc_venues_sorted
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_merged with df_toronto to add latitude/longitude, neighborhoods and boroughs
toronto_merged = toronto_merged.join(df_toronto.set_index('PostalCode'), on='PostalCode')

toronto_merged.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighborhood,Latitude,Longitude
0,M1B,Fast Food Restaurant,Print Shop,Yoga Studio,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,1,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Bar,Moving Target,History Museum,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,1,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Coffee Shop,Mexican Restaurant,Korean Restaurant,Garden Center,Deli / Bodega,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,1,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Lounge,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Yoga Studio,Department Store,1,Scarborough,Cedarbrae,43.773136,-79.239476


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, postal_code, borough, neighborhoods, cluster in zip(toronto_merged['Latitude'], 
                                                                  toronto_merged['Longitude'], 
                                                                  toronto_merged['PostalCode'],
                                                                  toronto_merged['Borough'],
                                                                  toronto_merged['Neighborhood'],
                                                                  toronto_merged['Cluster Labels']):
    label = '{}, {}, {}. Cluster {}'.format(neighborhoods, postal_code, borough, cluster + 1)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

##### 1st Cluster

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0, 13, 14, 1, 2, 3, 4, 5]]]

,PostalCode,Neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,M1V,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,Park,Playground,Coffee Shop,Dance Studio,Drugstore
22,M2P,York Mills West,43.752758,Park,Bank,Deli / Bodega,Eastern European Restaurant,Drugstore
24,M3A,Parkwoods,43.753259,Park,Fast Food Restaurant,Food & Drink Shop,Yoga Studio,Dance Studio
29,M3K,"CFB Toronto, Downsview East",43.737473,Park,Bus Stop,Airport,Drugstore,Dog Run
30,M3L,Downsview West,43.739015,Park,Shopping Mall,Grocery Store,Bank,Deli / Bodega
35,M4C,Woodbine Heights,43.695344,Park,Beer Store,Cosmetics Shop,Intersection,Dance Studio
39,M4J,East Toronto,43.685347,Park,Coffee Shop,Convenience Store,Dance Studio,Drugstore
43,M4N,Lawrence Park,43.728020,Bus Line,Swim School,Dim Sum Restaurant,Park,Concert Hall
49,M4W,Rosedale,43.679563,Park,Playground,Trail,Curling Ice,Drugstore
63,M5P,"Forest Hill North, Forest Hill West",43.696948,Jewelry Store,Park,Trail,Sushi Restaurant,Dance Studio


first cluster shows the parks location.

##### 2nd Cluster

In [36]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0, 13, 14, 1, 2, 3, 4, 5]]]

,PostalCode,Neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,"Rouge, Malvern",43.806686,Fast Food Restaurant,Print Shop,Yoga Studio,Dance Studio,Drugstore
1,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,Bar,Moving Target,History Museum,Yoga Studio,Deli / Bodega
2,M1E,"Guildwood, Morningside, West Hill",43.763573,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center
3,M1G,Woburn,43.770992,Coffee Shop,Mexican Restaurant,Korean Restaurant,Garden Center,Deli / Bodega
4,M1H,Cedarbrae,43.773136,Lounge,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Caribbean Restaurant
5,M1J,Scarborough Village,43.744734,Playground,Convenience Store,Yoga Studio,Dance Studio,Drugstore
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",43.727929,Discount Store,Coffee Shop,Bus Station,Chinese Restaurant,Train Station
7,M1L,"Clairlea, Golden Mile, Oakridge",43.711112,Bus Line,Park,Metro Station,Soccer Field,Intersection
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,American Restaurant,Motel,Yoga Studio,Deli / Bodega,Eastern European Restaurant
9,M1N,"Birch Cliff, Cliffside West",43.692657,General Entertainment,Café,Skating Rink,College Stadium,Comic Shop


Second cluster shows grocery, restaurants and lounges.

##### 3rd Cluster

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0, 13, 14, 1, 2, 3, 4, 5]]]

,PostalCode,Neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
31,M3M,Downsview Central,43.728496,Home Service,Baseball Field,Food Truck,Deli / Bodega,Eastern European Restaurant
90,M8Y,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,Home Service,Baseball Field,Yoga Studio,Eastern European Restaurant,Drugstore
95,M9M,"Emery, Humberlea",43.724766,Baseball Field,Construction & Landscaping,Yoga Studio,Department Store,Electronics Store


Third cluster shows the home service and baseball field.